Idea original modificada: http://philipperemy.github.io/keras-stateful-lstm/

Interestante:

https://machinelearningmastery.com/use-different-batch-sizes-training-predicting-python-keras/

## Generamos una secuencia artificial:
- Cada muestra es una secuencia binaria de 20 bits. 
- Para X: El primer bit es es 1 con una probabilidad de 0.5, el resto son todos ceros
- Para y: La etiqueta es 1 si el primer bit es 1

In [1]:
import numpy as np
N_samples = 1200
N_bits = 20
from numpy.random import choice
one_indexes = choice(a=N_samples, size=int(N_samples / 2), replace=False)
X = np.zeros((N_samples, N_bits))
X[one_indexes, 0] = 1  # very long term memory.
y = X[:,0]

In [2]:
N_train = 1000
X_train = X[:N_train]
y_train = y[:N_train]
X_validation = X[N_train:]
y_validation = y[N_train:]

In [3]:
idx = 1
print('Entrada: ',X_train[idx])
print('Salida:', y_train[idx])

Entrada:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Salida: 0.0


In [4]:
print('Training')
print(X_train.shape)
print(y_train.shape)
print('Validation')
print(X_validation.shape)
print(y_validation.shape)

Training
(1000, 20)
(1000,)
Validation
(200, 20)
(200,)


In [5]:
def prepare_sequences(x_train, y_train, window_length):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    return np.array(windows), np.array(windows_y)

## Que pasa si partimos la secuencia para entrenar?

### Probar:
- window_length = 10, 15, 20
- batch_size = 1, 50, 1000

In [6]:
window_length = 20
X_train_split_1, y_train_split_1 = prepare_sequences(X_train, y_train, window_length)
X_validation_split_1, y_validation_split_1 = prepare_sequences(X_validation, y_validation, window_length)

In [7]:
# Empieza el nuevo vector en (N_bits - window_length + 1)
next_vect_delta = N_bits - window_length + 1
print(X_train_split_1[next_vect_delta])
# Los y's son todos unos si el primer valor del vector fue 1
print(y_train_split_1[next_vect_delta])
print(y_train_split_1[next_vect_delta+1])
print(X_train_split_1.shape)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.0
1.0
(1000, 20)


In [8]:
from keras.layers import SimpleRNN, Dense, LSTM
from keras.models import Sequential
batch_size = 50

model = Sequential()
model.add(LSTM(10, input_shape=(window_length, 1), return_sequences=False, stateful=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_split_1.reshape(-1,window_length,1), y_train_split_1, batch_size=batch_size, epochs=15,
          validation_data=(X_validation_split_1.reshape(-1,window_length,1), y_validation_split_1), shuffle=False)
score, acc = model.evaluate(X_validation_split_1.reshape(-1,window_length,1), y_validation_split_1, batch_size=1, verbose=1)
print(score, acc)

Using TensorFlow backend.


Train on 1000 samples, validate on 200 samples
Epoch 1/15
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6925 - acc: 0.5210 - val_loss: 0.6919 - val_acc: 0.4800
Epoch 2/15
1000/1000 [==============================] - 1s 548us/step - loss: 0.6902 - acc: 0.5040 - val_loss: 0.6875 - val_acc: 0.4800
Epoch 3/15
1000/1000 [==============================] - 0s 475us/step - loss: 0.6765 - acc: 0.5040 - val_loss: 0.6540 - val_acc: 0.4800
Epoch 4/15
1000/1000 [==============================] - 0s 429us/step - loss: 0.5659 - acc: 0.5790 - val_loss: 0.4221 - val_acc: 1.0000
Epoch 5/15
1000/1000 [==============================] - 0s 439us/step - loss: 0.3010 - acc: 1.0000 - val_loss: 0.2318 - val_acc: 1.0000
Epoch 6/15
1000/1000 [==============================] - 0s 421us/step - loss: 0.1932 - acc: 1.0000 - val_loss: 0.1711 - val_acc: 1.0000
Epoch 7/15
1000/1000 [==============================] - 0s 441us/step - loss: 0.1459 - acc: 1.0000 - val_loss: 0.1302 - val_acc: 1.0000
Epo

- Como cambia la velocidad con los distintos tamaños de batch?
- Estudiar tiempos de convergencia
- Converge para todos los casos de longitud de la secuencia?

## Usando Batch_size de 1, stateful y window_lenght = 1

In [9]:
window_length = 1
X_train_split_2, y_train_split_2 = prepare_sequences(X_train, y_train, window_length)
X_validation_split_2, y_validation_split_2 = prepare_sequences(X_validation, y_validation, window_length)

In [10]:
batch_size = 1

model = Sequential()
model.add(LSTM(10, batch_input_shape=(1, 1, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train_split_2.reshape(-1,window_length,1), y_train_split_2, batch_size=batch_size, epochs=1,
          validation_data=(X_validation_split_2.reshape(-1,window_length,1), y_validation_split_2), shuffle=False)
score, acc = model.evaluate(X_validation_split_2.reshape(-1,window_length,1), y_validation_split_2, batch_size=1, verbose=1)
print(score, acc)

Train on 20000 samples, validate on 4000 samples
Epoch 1/1
4000/4000 [==============================] - 5s 1ms/step
0.6974221825525164 0.52


Que pasa en este caso? Por que no llega a mejorar la accuracy?

## Reseteo despues de cada secuencia

In [11]:
# N_bits es 20, la longitud de la secuencia
i = 0
X_validation_split_2[i*N_bits:(i+1)*N_bits]

array([[1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [12]:
batch_size = 1

model = Sequential()
model.add(LSTM(10, batch_input_shape=(1, 1, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

for i in range(X_train_split_2.shape[0]):
    print(i)
    model.fit(X_train_split_2[i*N_bits:(i+1)*N_bits].reshape(-1,window_length,1), 
              y_train_split_2[i*N_bits:(i+1)*N_bits], batch_size=batch_size, epochs=1, shuffle=False, verbose=1)
    model.reset_states()
score, acc = model.evaluate(X_validation_split_2.reshape(-1,window_length,1), y_validation_split_2, batch_size=1, verbose=1)
print(score, acc)

0
Epoch 1/1
20/20 [==============================] - 1s 27ms/step - loss: 0.6805 - acc: 0.8000
1
Epoch 1/1
20/20 [==============================] - 0s 4ms/step - loss: 0.7370 - acc: 0.0000e+00
2
Epoch 1/1
20/20 [==============================] - 0s 4ms/step - loss: 0.6671 - acc: 0.9000
3
Epoch 1/1
20/20 [==============================] - 0s 4ms/step - loss: 0.7316 - acc: 0.0000e+00
4
Epoch 1/1
20/20 [==============================] - 0s 4ms/step - loss: 0.7012 - acc: 0.2000
5
Epoch 1/1
20/20 [==============================] - 0s 4ms/step - loss: 0.7044 - acc: 0.0000e+00
6
Epoch 1/1
20/20 [==============================] - 0s 4ms/step - loss: 0.6886 - acc: 1.0000
7
Epoch 1/1
20/20 [==============================] - 0s 4ms/step - loss: 0.6643 - acc: 1.0000
8
Epoch 1/1
20/20 [==============================] - 0s 4ms/step - loss: 0.6238 - acc: 1.0000
9
Epoch 1/1
20/20 [==============================] - 0s 4ms/step - loss: 0.5510 - acc: 1.0000
10
Epoch 1/1
20/20 [===========================

AttributeError: 'ProgbarLogger' object has no attribute 'log_values'

## Batch size = 1000 y reshape del dataset

In [13]:
batch_size = 1000
X_train_split, y_train_split = prepare_sequences(X_train, y_train, 1)
N = len(X_train_split)
frac = N/batch_size
print(N, frac)
X_train_split_reshaped = X_train_split.reshape(batch_size,int(frac)).T.reshape(N,1,1)
y_train_split_reshaped = y_train_split.reshape(batch_size,int(frac)).T.reshape(N)
print(X_train_split.shape)
print(X_train_split_reshaped.shape)

20000 20.0
(20000, 1)
(20000, 1, 1)


In [14]:
from keras.layers import SimpleRNN, Dense, LSTM
from keras.models import Sequential
model = Sequential()
model.add(LSTM(10, batch_input_shape=(batch_size, 1, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
for i in range(N_bits):
    model.fit(X_train_split_reshaped, y_train_split_reshaped, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
    model.reset_states()

Epoch 1/1
20000/20000 [==============================] - 1s 33us/step - loss: 0.6890 - acc: 0.7496
Epoch 1/1
20000/20000 [==============================] - 0s 4us/step - loss: 0.6879 - acc: 0.5040
Epoch 1/1
20000/20000 [==============================] - 0s 4us/step - loss: 0.6865 - acc: 0.6760
Epoch 1/1
20000/20000 [==============================] - 0s 4us/step - loss: 0.6844 - acc: 0.7252
Epoch 1/1
20000/20000 [==============================] - 0s 4us/step - loss: 0.6808 - acc: 0.7504
Epoch 1/1
20000/20000 [==============================] - 0s 4us/step - loss: 0.6727 - acc: 0.9008
Epoch 1/1
20000/20000 [==============================] - 0s 4us/step - loss: 0.6451 - acc: 0.9752
Epoch 1/1
20000/20000 [==============================] - 0s 4us/step - loss: 0.5218 - acc: 1.0000
Epoch 1/1
20000/20000 [==============================] - 0s 4us/step - loss: 0.3909 - acc: 1.0000
Epoch 1/1
20000/20000 [==============================] - 0s 4us/step - loss: 0.2670 - acc: 1.0000
Epoch 1/1
20000/200

## Para poder evaluar tengo que recargar el modelo con batch_size=200 para que sea eficiente

In [16]:
X_val_split, y_val_split = prepare_sequences(X_validation, y_validation, 1)
X_val_split_reshaped = X_val_split.reshape(200,20).T.reshape(4000,1,1)
y_val_split_reshaped = y_val_split.reshape(200,20).T.reshape(4000)

In [17]:
new_model = Sequential()
new_model.add(LSTM(10, batch_input_shape=(200, 1, 1), return_sequences=False, stateful=True))
new_model.add(Dense(1, activation='sigmoid'))
new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
old_weights = model.get_weights()
new_model.set_weights(old_weights)

In [19]:
new_model.evaluate(X_val_split_reshaped, y_val_split_reshaped, batch_size=200, verbose=1)

4000/4000 [==============================] - 0s 51us/step


[0.07798209129832685, 1.0]

# Batch size = 100

Hay que tomar de a 100 del total de 1000, hacer un reshape a (100,20) y luego trasponer

In [20]:
X_train_split, y_train_split = prepare_sequences(X_train, y_train, 1)

In [21]:
# Asi es como veria los datos el modelo de training
X_train_split.reshape(1000,20)[:100].T #.flatten()

array([[1., 0., 1., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
X_train_split.shape

(20000, 1)

In [23]:
X_train_split_reshaped = np.zeros((X_train_split.shape[0]))

In [24]:
batch_size = 100
model = Sequential()
model.add(LSTM(10, batch_input_shape=(batch_size, 1, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
n_batches = int(len(X_train_split_reshaped)/batch_size)
print(n_batches)
for k in range(20):
    i = k%10
    X_to_train = (X_train_split.reshape(1000,20)[i*batch_size:(i+1)*batch_size].T.flatten()).reshape(2000,1,1)
    y_to_train = y_train_split.reshape(1000,20)[i*batch_size:(i+1)*batch_size].T.flatten()
    model.fit(X_to_train, y_to_train, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
    model.reset_states()

200
Epoch 1/1
2000/2000 [==============================] - 1s 406us/step - loss: 0.6949 - acc: 0.5600
Epoch 1/1
2000/2000 [==============================] - 0s 37us/step - loss: 0.7007 - acc: 0.4200
Epoch 1/1
2000/2000 [==============================] - 0s 38us/step - loss: 0.6943 - acc: 0.5660
Epoch 1/1
2000/2000 [==============================] - 0s 38us/step - loss: 0.6932 - acc: 0.4845
Epoch 1/1
2000/2000 [==============================] - 0s 40us/step - loss: 0.6922 - acc: 0.4845
Epoch 1/1
2000/2000 [==============================] - 0s 39us/step - loss: 0.6922 - acc: 0.4655
Epoch 1/1
2000/2000 [==============================] - 0s 37us/step - loss: 0.6908 - acc: 0.4560
Epoch 1/1
2000/2000 [==============================] - 0s 41us/step - loss: 0.6863 - acc: 0.6700
Epoch 1/1
2000/2000 [==============================] - 0s 38us/step - loss: 0.6953 - acc: 0.4100
Epoch 1/1
2000/2000 [==============================] - 0s 41us/step - loss: 0.6838 - acc: 0.7240
Epoch 1/1
2000/2000 [====

In [26]:
X_val_split, y_val_split = prepare_sequences(X_validation, y_validation, 1)
X_val_split_reshaped = X_val_split.reshape(200,20).T.reshape(4000,1,1)
y_val_split_reshaped = y_val_split.reshape(200,20).T.reshape(4000)

new_model = Sequential()
new_model.add(LSTM(10, batch_input_shape=(200, 1, 1), return_sequences=False, stateful=True))
new_model.add(Dense(1, activation='sigmoid'))
new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

old_weights = model.get_weights()
new_model.set_weights(old_weights)

new_model.evaluate(X_val_split_reshaped, y_val_split_reshaped, batch_size=200, verbose=1)

4000/4000 [==============================] - 0s 56us/step


[0.1347997548058629, 1.0]